In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "white",color_codes=True)
sns.set(font_scale=1.5)
from IPython.display import display
pd.options.display.max_columns = None

from sklearn.model_selection import GridSearchCV #to fine tune Hyperparamters using Grid search
from sklearn.model_selection import RandomizedSearchCV# to seelect the best combination(advance ver of Grid Search)

# importing some ML Algorithms 
from sklearn.linear_model import LinearRegression # y=mx+c
from sklearn.tree import DecisionTreeRegressor # Entropy(impurities),Gain. 
from sklearn.ensemble import RandomForestRegressor # Average of Many DT's

# Testing Libraries - Scipy Stats Models
from scipy.stats import shapiro # Normality Test 1
from scipy.stats import normaltest # Normality Test 2
from scipy.stats import anderson # Normality Test 3
from statsmodels.graphics.gofplots import qqplot # plotting the Distribution of Y with a Line of dot on a 45 degree Line.

# Model Varification/Validation Libraries
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit


# Matrices and Reporting Libraries
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import make_scorer
from statsmodels.tools.eval_measures import rmse
from sklearn.model_selection import learning_curve


In [ ]:
ds= pd.read_csv("../input/mercedesbenz-greener-manufacturing/train.csv")
dt = pd.read_csv("../input/mercedesbenz-greener-manufacturing/test.csv")

In [ ]:
ds.head()

### Alright, Lets do some Data Exploratory Visualisation

In [ ]:
ds.columns , dt.columns

In [ ]:
ds.shape , dt.shape

In [ ]:
ds.dtypes

In [ ]:
ds.select_dtypes(include='float').columns

In [ ]:
ds.select_dtypes(include='int64').columns

In [ ]:
ds.select_dtypes(include='object').columns

Some common Obsevations:
- we can see that the columns X0 to X8 (excudes X7) are categorical 
- Target column 'y' is floating value. 
- Other columns are off int type.

we can observe the columns X10 till the end has values 0 and 1. Let us confirm if these values are correct.

In [ ]:
# training Data
np.unique(ds[ds.columns[10:]])

In [ ]:
# testomg Data
np.unique(dt[dt.columns[10:]])

this confimrs that there are only 0 and 1 from the column with index 10 till end. Which is X10 to X385

#### Finding the Null Values. 

In [ ]:
#training 
ds.isnull().sum().unique()

In [ ]:
# testing
dt.isnull().sum().unique()

confirmed that there are no Nulls. 

#### Lets see how the Target column is distributed in Training Data. 

In [ ]:
ds["y"].describe()

IQR looks like there might be outliers on the Higher values. But this we will check in the Outliers section. 

#### Plotting the Y - Since this column is Time representation, lets see how the time is Distributed using different Plots. 

In [ ]:
ds.shape

In [ ]:
fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(16,6))
fig.suptitle("Distribution of Vaehicle Testing Times",fontsize=12)

ax1.scatter(range(ds.shape[0]),np.sort(ds["y"].values)) 
ax2.plot(ds["y"])
sns.distplot(ds["y"],kde=True,ax=ax3,bins=70)
qqplot(ds["y"],line='s',ax=ax4)

Observations : 

- Target Y column is almost normally distributed from the KDE grap we can see that. However, there are spikes intermitently. 
- Quantiles Plot with Slope Line shows that the deviation from the expected time. We can notice more deviation are the starting values and at the end. 
- THe Data is Positively Skewed. So the Outliers might be at the End/last quartile. ANd We can notice a single point which is outlined from the Qualtile slope line.

### Normality Test
Lets Perform some Normality Test for proving the Hypothesis.

- Null Hyp - H0 is that , a sample was drawn from the Normal Distribution. (Accept the H0/Failed To Reject H0)
- And if P=value > 50% then the Test failed to reject the H0.

In [ ]:
# test 1 - Shapiro Test 
stat , p = shapiro(ds["y"])
print('Stat = %.3f, p = %.3f'%(stat,p))

if p>0.05:
    print('Sampe looks like Gausian or Sample is drawn from it.(Failed TO reject the H0)')
else:
    print('Reject the H0. THis is not Noemaly Distributed')

In [ ]:
# test 1 - K2 test 
stat , p = normaltest(ds["y"])
print('Stat = %.3f, p = %.3f'%(stat,p))

if p>0.05:
    print('Sampe looks like Gausian or Sample is drawn from it.(Failed TO reject the H0)')
else:
    print('Reject the H0. THis is not Noemaly Distributed')

In [ ]:
# Anderson Test
st,cv,sl= anderson(ds["y"])
for a,b in zip(cv,sl):
    if st < a:
        print("{}   {}      {}".format())
        print("{} {} {}(accept H0)".format(a,b,st))
    else:
        print('{} {}% {:.2f}(reject H0)'.format(a,int(b),st))


### Analyzing the Columns in Training and Testing Data
THis will help us to compare the training and testing Data. Both should Match. 

In [ ]:
# Training
print(ds.select_dtypes(include='object').columns)
print(ds.select_dtypes(include='float').columns)
print(ds.select_dtypes(include='int64').columns)

In [ ]:
# Testing
print(dt.select_dtypes(include='object').columns)
print(dt.select_dtypes(include='float').columns)
print(dt.select_dtypes(include='int64').columns)

### Looking unique values of each feature/column in Training and Testing


In [ ]:
# Lets Tackkle the Object columns to see if the same values are there in Testing data also, or anything new 
t = ds.select_dtypes(include='object').columns
tt = dt.select_dtypes(include='object').columns
for a,b in zip(t,tt):
    print('\nunique values in '+a+' for Training sample are : {}  '.format(ds[a].nunique()))
    print('unique values in '+b+' for Testing samples are : {}  '.format(dt[b].nunique()))
    print('values that are available in '+a+' Training but not in '+b+' Testing  : {}'.format(list(set(ds[a]).difference(dt[b]))))
    print('values that are available in '+b+' Testing but not in '+a+' Training : {}'.format( list(set(dt[b]).difference(ds[a]))))



observations :

    As you can see the data in training and Testing are not the same. So we need to combine these to DS and get the Dummies out off it. So that we dont miss out any new Data. Any new Data in testing sample will make our ML model confused.

comparing the testing Target Y with the the categorical Columns so that we can discrimiate the Variance. Columns that are not discrimiatingcan be removed.

Note: We are doing this to the Testing DS and not the Training DS.


In [ ]:
for col in tt:
    plt.figure(figsize=(16,6))
    sns.boxplot(x=col,y='y',data=ds)
    plt.xlabel(col,fontsize=10)
    plt.ylabel('y',fontsize=10)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=10)

#### Observations

- X0,X1 and X2 seems to be well discriminating.Why because the Variance is high and they are good predictors for testing.
- X3,X4,X5 and X6 doesnt seems to be well discriminating power.These Columns can be removed

### Feature Engineering - To check if Categorical Variables X0 X1, X2 ...X8 has strong relationship with Y- Time

In [ ]:
#### ANOVA test - Since input is Categorical and Output is Numerical.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
model = ols ('y ~ C(X4)',data=ds).fit()    
print('F-Statics: ',model.fvalue)
print('p-value: ',model.f_pvalue)

In [ ]:
anova_table = sm.stats.anova_lm(model,type=2)
anova_table

observation : 
- Here in this test alone says p < 0.05 and the H0 can be rejected. 
- THis means there is no significant difference netween the weights.
- Here the X4 column can be retained. 

#### Testing Anova using the Pair Wise Test since the previous P-value is close to 0.05

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
m_comp = pairwise_tukeyhsd(endog=ds['y'],groups=ds['X4'],alpha=0.05)
print(m_comp)

observations:

    Here the Null hypothesis has failed to reject. That means we are accepting the H0. Which is a diffrent result compared to previous one. THat means there is no difference in the weights . THat also means we can drop the column X4.



#### Lets test the Anova Test for all the columns. In that wasy we can see weather the column X can be retained or dropped. 

In [ ]:
columns = t.values
columns

for col in columns: 
    model = ols('y ~ '+col, data=ds).fit()
    print('Column {}, F-stat: {:.2f} , F-Pvalues {:.2f}'.format(col,model.fvalue,model.f_pvalue))

Here the X4 has P-Value >= 0.05 , That means the H0 has failed to Reject. THat means Sample was drawn from the Gausian distribution. THat also means that there is no Discrimiation. THat means the column is similar. Meaning, it might be dependednt/ related . Related column cannot be considered as the entropy will be high. So we need to drop the column.

### Lets detecct Outliers

In [ ]:
# calculating the 25% percentile
Q1 = np.percentile(ds.loc[:,'y'],25) # fancy way of slicing only the Y column. You can also use ds["y"]

# calculating the 50% percentile
Q2 = np.percentile(ds.loc[:,'y'],50)

# calculating the 75% percentile
Q3 = np.percentile(ds.loc[:,'y'],75)

# Calculating the outlier using the IQR (Q3-Q1) and 1.5 (left over 1% in normal Distribution curve)
step = (Q3-Q1) *3 

print('Q1 = {}, Q2= {}, Q3={} , Outlier Step = {}'.format(Q1,Q2,Q3,step))

In [ ]:
Left_side_outlier = ds[ds['y'] <= (Q1-step)].index  # Int64Index([], dtype='int64') / no outliers on Left side
Right_side_outlier = ds[ds['y'] > (Q3+step)].index # Int64Index([342, 883, 1459, 3133], dtype='int64') 4 indexs of Outliers on Right side

In [ ]:
num_left_outlier = len(Left_side_outlier) # 0
num_right_outlier = len(Right_side_outlier) # 4 

print('Number of the Outliers on the lower Side = {}'.format(num_left_outlier))
print('Number of the Outliers on the upper Side = {}'.format(num_right_outlier))

print('lower outliers = {}%'.format((num_left_outlier/ds.shape[0])*100)) #(0/4209) * 100 - this is like accuracy, Number of errors(outliers) / total number of elements(len of Y column). 
print('lower outliers = {}%'.format((num_right_outlier/ds.shape[0])*100)) #( 4/4209) * 100

In [ ]:
ds.iloc[Right_side_outlier]['y'].sort_values(ascending= False) # Slicing those outliers Index from the DS to see their Y values.

In [ ]:
# Only one Outlier 
ds[ds['y'] >= 170]

## converting categorelical Columns to numerical 

###  Approach 1 - One Hot encoding
- Here we first combine the Training and testing data because if there are any unique alues in testing data then the model will fail to identify or perdict. 
- Then we will get Dummies to convert the categorical columns to dummies.

In [ ]:
combined_data = ds.append(dt,ignore_index=True)
combined_data = pd.get_dummies(combined_data)

In [ ]:
ds.shape , dt.shape , combined_data.shape

#### Splitting the Feature and target columns of the combined DS

In [ ]:
train,test = combined_data[0:len(ds)],combined_data[len(dt):] # train 0-4209 ,  Test 4209-8418

In [ ]:
train.shape,test.shape

In [ ]:
x_feature = train.drop(['y','ID'],axis=1)
y_target = train['y']

In [ ]:
X_train_features,X_test_features,Y_train_target,Y_test_target = train_test_split(x_feature,y_target,test_size= 0.25 , random_state=4)

In [ ]:
X_train_features.shape, Y_train_target.shape , X_test_features.shape,Y_test_target.shape

## Performing  - 
### 1 . Linear Regression

In [ ]:
# Instantiate
lin = LinearRegression()


In [ ]:
%%time
lin.fit(X_train_features,Y_train_target) # Fit the Data
# Prediction - Training Samples
y_pred = lin.predict(X_train_features)

print('\nTraining Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_train_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_train_target,y_pred))

# Prediction - Testing Samples
y_pred = lin.predict(X_test_features)

print('\nTesting Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_test_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_test_target,y_pred))

## Performing  - 
### 2 . Random Forest - RF

In [ ]:
# instantiate
rf = RandomForestRegressor(n_estimators = 20, #Number of decision trees used by the RF
                           criterion='mse',# method to calculate the quality of split
                          max_depth= 4 , # Max depth of Each Decision Tree
                          max_features='auto', 
#say for example We have couple of DT's with 40 columns each, lets take Max feature =8 . 
#Then we calculate the Entropy to all the 40 columns and take the Mean and Max information Gain. This process is common in DT. 
#But here, in Max feature, we will take random heighest features of 8 to decide the Max info Gain. That’s all the difference. Why we are doing this is to choose the averaging. Net effect of this will give you Unbaisad output.

                           min_samples_split = 0.05
                           )

In [ ]:
%%time
rf.fit(X_train_features,Y_train_target) # Fit the Data
# Prediction - Training Samples
y_pred = rf.predict(X_train_features)

print('\nTraining Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_train_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_train_target,y_pred))

# Prediction - Testing Samples
y_pred = rf.predict(X_test_features)

print('\nTesting Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_test_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_test_target,y_pred))

#### Observation . RF is little better than Linear Regression because here we have R2 = 58 and 57 unlike LR whic has -R2 for testing samples.
 R2 = 0.58. This implies that 58% of the variability of the dependent variable has been accounted for, and the remaining 42% of the variability is still unaccounted for. So R2 should be close to 1

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# instantiate
knn = KNeighborsRegressor()
#fit
knn.fit(X_train_features,Y_train_target) # Fit the Data
# Prediction - Training Samples
y_pred = knn.predict(X_train_features)

print('\nTraining Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_train_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_train_target,y_pred))

# Prediction - Testing Samples
y_pred = rf.predict(X_test_features)

print('\nTesting Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_test_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_test_target,y_pred))

###### KNN is almost same as the RF

### 4. Gradiant Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# instantiate
gbm = GradientBoostingRegressor(n_estimators=600,
                               max_depth=8,
                               min_samples_split=.1,
                               max_features='sqrt',
                               learning_rate=0.01,
                               loss='ls')
#fit
gbm.fit(X_train_features,Y_train_target) # Fit the Data
# Prediction - Training Samples
y_pred = gbm.predict(X_train_features)

print('\nTraining Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_train_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_train_target,y_pred))

# Prediction - Testing Samples
y_pred = rf.predict(X_test_features)

print('\nTesting Score : ')
print('Mean Squared Error = %.2f'% mean_squared_error(Y_test_target,y_pred))
print('R2 score : %.2f'% r2_score(Y_test_target,y_pred))

### 5. XGB Regression Model
- we split the training and Testing data.
- Load the data in dtrain and dtest , which is the binary representation and is xtremely fast for XGB
- THen take the mean of the Target column of the training Data. 
- Then Get the Base line Prediction for the Testing Target column.BLP = (Testing target.shape) * (Mean of the Training Target)
- Find the MAE for the above. mean_absolute_error(test_target,BLP)
- Ajenda is to reach teh MAE as the Base line MAE. 
- Train the Midel with XGB parameters. Model.
- Use this Model to Predict the Dtrain and Dtest data and their scores. 
- Compare these scores with the Base Line MAE.

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [ ]:
X_train_features,X_test_features,Y_train_target,Y_test_target = train_test_split(x_feature,y_target,test_size= 0.2 , random_state=45)

In [ ]:
# creating DMatrices for the XGB training. This is a Binary Representation
dtrain = xgb.DMatrix(X_train_features,label = Y_train_target)
dtest = xgb.DMatrix(X_test_features,label = Y_test_target)

In [ ]:
mean_train = Y_train_target.mean()

In [ ]:
# Get prediction on the test set
baseline_predictions  = np.ones(Y_test_target.shape)*mean_train

In [ ]:
baseline_predictions

In [ ]:
mae_baseline = mean_absolute_error(Y_test_target,baseline_predictions)
print("BaseLine Mae is : %.2f"%mae_baseline)

In [ ]:
xgb_param = { "max_depth" : 8,
             "min_child_weight":1,
             'eta':.35,
             'subsample':1,
             'colsample_bytree':.9,
             'objective':'reg:squarederror',
             'eval_metric':'mae',
             'validate_parameters':1,
             'verbose_eval':False
            }

In [ ]:
%%time
model = xgb.train(xgb_param,
                 dtrain,
                 num_boost_round=999,
                 evals=[(dtest,'Test')],
                 early_stopping_rounds=10
                 )

In [ ]:
# predictions
y_pred=model.predict(dtrain) # adding the DMetrics Binary Values of Training Data.

print("Training : Metrics...")
print("Mean ABS Error MAE :     ",metrics.mean_absolute_error(Y_train_target,y_pred))
print("Mean sq Error MSE :      ",metrics.mean_squared_error(Y_train_target,y_pred))

print('Root Mean Sq Error RMSE: ',np.sqrt(metrics.mean_squared_error(Y_train_target,y_pred)))
print('r2value      :           ',metrics.r2_score(Y_train_target,y_pred))

y_pred = model.predict(dtest) # adding the DMetrics Binary Values of Testing Data.

print('\nTesting : Metrics ... ')
print('Mean Abs Error MAE :     ',metrics.mean_absolute_error(Y_test_target,y_pred))
print("Mean sq Error MSE :      ",metrics.mean_squared_error(Y_test_target,y_pred))

print('Root Mean Sq Error RMSE: ',np.sqrt(metrics.mean_squared_error(Y_test_target,y_pred)))
print('r2value      :           ',metrics.r2_score(Y_test_target,y_pred))

### Observations : XGB is the best among all for this Model. 
